In [2]:
# pip install peft transformers datasets accelerate bitsandbytes

In [35]:
from datasets import load_dataset
import re
import random
from tqdm import tqdm
import numpy as np

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from peft import prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, balanced_accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Используемое устройство: cuda


In [36]:
def sample_data(dataset, num_samples_per_class, labels):
    sampled_data = []
    for label in range(len(labels)):
        class_data = [example for example in dataset if example["label"] == label]
        sampled_data.extend(random.sample(class_data, num_samples_per_class))
        random.shuffle(sampled_data)
    return sampled_data

def format_instruction(example):
    instruction = """
    Instruction: Determine the category of the given text (provided below).
    Choose exactly one category from the following options: World, Sports, Business, or Sci/Tech.
    Your output should be a single word representing the selected category."""
    text = example["text"]
    output = labels[example["label"]]
    return {
        "instruction": instruction,
        "text": text,
        "output": output
    }

def make_instruction(dct):
  return f"""
  {dct['instruction']}

    Text: {dct['text']}

    Answer: {dct['output']}"""

def make_instruction_test(dct):
  return f"""
  {dct['instruction']}

    Text: {dct['text']}

    Answer:"""

class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = {key: val.squeeze(0) for key, val in self.data[idx].items()}
        # Копируем input_ids как метки (если это задача языкового моделирования)
        item['labels'] = item['input_ids'].clone()
        return item

def extract_answer(prediction, valid_categories):
    """
    Извлекает категорию из текста предсказания модели с учетом нижнего регистра.

    :param prediction: Текст предсказания
    :param valid_categories: Список допустимых категорий
    :return: Извлечённая категория или None, если не удалось найти валидное значение
    """
    # Приведение допустимых категорий к нижнему регистру
    normalized_categories = [cat.lower() for cat in valid_categories]

    # Ищем слово после "Answer:"
    match = re.search(r"Answer:\s*([\w/]+)", prediction)
    if match:
        extracted = match.group(1).strip().lower()  # Приводим к нижнему регистру
        # Проверяем, является ли это слово валидной категорией
        if extracted in normalized_categories:
            return extracted  # Возвращаем в нижнем регистре
    # Если не удалось найти валидное слово, возвращаем первую найденную категорию
    for category in normalized_categories:
        if category in prediction.lower():
            return category
    return None

## Загрузка AG News

In [37]:
labels = ["World", "Sports", "Business", "Sci/Tech"]

In [38]:
dataset = load_dataset("ag_news")

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [39]:
train_samples = sample_data(dataset["train"], 400, labels)
test_samples = sample_data(dataset["test"], 100, labels)

formatted_train = [format_instruction(sample) for sample in train_samples]
formatted_test = [format_instruction(sample) for sample in test_samples]

# save_to_file(formatted_train, "small_train_data.txt")
# save_to_file(formatted_test, "small_test_data.txt")

In [40]:
print(make_instruction(formatted_train[13]))


  
    Instruction: Determine the category of the given text (provided below).
    Choose exactly one category from the following options: World, Sports, Business, or Sci/Tech.
    Your output should be a single word representing the selected category.

    Text: Search Engine Milestones for August 2004 Notable news and announcements from the web search world during the past month. 

    Answer: Sci/Tech


In [41]:
print(make_instruction_test(formatted_test[13]))


  
    Instruction: Determine the category of the given text (provided below).
    Choose exactly one category from the following options: World, Sports, Business, or Sci/Tech.
    Your output should be a single word representing the selected category.

    Text: Mood Mixed Among Darfur Rebels Ahead of Talks CORCHA CAMP, Sudan (Reuters) - A Sudanese rebel commander in a camp in Darfur tells his troops he is hoping for peace. But just a few hours march away, young men say they are convinced Sudan wants to drive them off the land. 

    Answer:


## Model downloading

In [42]:
# model_name = "EleutherAI/gpt-neo-125m"
model_name = "EleutherAI/gpt-neo-1.3B"
# model_name = "EleutherAI/gpt-neo-2.7B"
# model_name = "EleutherAI/gpt-j-6b"
# model_name = "EleutherAI/gpt-neox-20b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [43]:
# print(f"Модель находится на устройстве: {next(model.parameters()).device}")

In [44]:
# Установка токена для заполнения
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("<PAD>")

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

max_length=256

# Токенизация данных
tokenized_train = [tokenizer(
    make_instruction(item),
    truncation=True,
    padding="max_length",
    return_tensors="pt",
    max_length=max_length
) for item in formatted_train]

tokenized_test = [tokenizer(
    make_instruction_test(item),
    truncation=True,
    padding="max_length",
    return_tensors="pt",
    max_length=max_length
) for item in formatted_test]

In [45]:
# tokenizer.model_max_length

In [46]:
# # formatted_string = make_instruction()
# # inputs = tokenizer(formatted_string, return_tensors="pt", truncation=True, padding=True)
# inputs = tokenized_test[0]
# inputs = {key: val.to(device) for key, val in inputs.items()}
# print(formatted_test[0])

In [47]:
# outputs = model(**inputs)
# logits = outputs.logits

# # print(logits)

# predicted_token_ids = torch.argmax(logits, dim=-1)
# decoded_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
# # print(decoded_text)

## Model Training

In [48]:
def calculate_max_steps(num_samples, num_epochs, batch_size, grad_accumulation_steps):
    effective_batch_size = batch_size * grad_accumulation_steps
    steps_per_epoch = num_samples // effective_batch_size
    max_steps = steps_per_epoch * num_epochs
    return max_steps

num_samples = 1600
num_epochs = 2
batch_size = 2
grad_accumulation_steps = 4

max_steps = calculate_max_steps(num_samples, num_epochs, batch_size, grad_accumulation_steps)
print(f"Max steps: {max_steps}")

Max steps: 400


In [49]:
# for name, module in model.named_modules():
#     print(name)

In [50]:
# ["query_key_value"] 20 b
# ["q_proj", "v_proj", "out_proj"] 6b
# ["c_attn", "c_proj"] < 6b

lora_config = LoraConfig(
    task_type="CAUSAL_LM",  # Задача каузального языкового моделирования
    r=16,                         # Ранг матрицы LoRA
    lora_alpha=32,                # Коэффициент масштабирования
    lora_dropout=0.05,             # Дропаут LoRA
    target_modules=["c_attn", "c_proj"],  # Модули внимания
    bias="none"
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,319,510,016 || trainable%: 0.2980


In [51]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=400,
    warmup_steps=40,
    learning_rate=5e-4,
    fp16=True,
    logging_steps=5,
    output_dir="outputs",
    optim="paged_adamw_8bit",
    # num_train_epochs=3,
    # lr_scheduler_type='linear'
)


trainer = Trainer(
    model=peft_model,
    train_dataset=CustomDataset(tokenized_train),
    eval_dataset=CustomDataset(tokenized_test),
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [52]:
# 741bdface455df61c3a81a9b26fbf9e8e1703a69

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
5,12.807600
10,12.676300
15,11.755700
20,10.370500
25,8.593500
30,7.301000
35,6.194800
40,5.745000
45,5.601800
50,5.226200


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=400, training_loss=5.328694767951966, metrics={'train_runtime': 1189.6762, 'train_samples_per_second': 2.69, 'train_steps_per_second': 0.336, 'total_flos': 5959126194585600.0, 'train_loss': 5.328694767951966, 'epoch': 2.0})

## Evaluation

In [53]:
# model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
# model_to_save.save_pretrained("news_pretrained")

In [54]:
# lora_config = LoraConfig.from_pretrained('news_pretrained')
# model = get_peft_model(model, lora_config)

In [61]:
i = 0

text = make_instruction_test(formatted_test[i])

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = peft_model.generate(**inputs, max_new_tokens=5, pad_token_id=tokenizer.pad_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))


  
    Instruction: Determine the category of the given text (provided below).
    Choose exactly one category from the following options: World, Sports, Business, or Sci/Tech.
    Your output should be a single word representing the selected category.

    Text: IBM #39;s New eServer Supports AMD Dual-Core IBM (Quote, Chart) is looking to get a leg up on the competition with the October 15 launch of eServer 326, a rack-mounted server that supports AMD #39;s (Quote, Chart) upcoming dual-core 64-bit processor.

    Answer: Sci/Tech IBM #


In [62]:
formatted_test[i]

{'instruction': '\n    Instruction: Determine the category of the given text (provided below).\n    Choose exactly one category from the following options: World, Sports, Business, or Sci/Tech.\n    Your output should be a single word representing the selected category.',
 'text': 'IBM #39;s New eServer Supports AMD Dual-Core IBM (Quote, Chart) is looking to get a leg up on the competition with the October 15 launch of eServer 326, a rack-mounted server that supports AMD #39;s (Quote, Chart) upcoming dual-core 64-bit processor.',
 'output': 'Sci/Tech'}

In [57]:
list_resp = []
for i in tqdm(range(len(formatted_test))):
  text = make_instruction_test(formatted_test[i])

  inputs = tokenizer(text, return_tensors="pt").to(device)
  outputs = peft_model.generate(**inputs,
                                max_new_tokens=5,
                                pad_token_id=tokenizer.pad_token_id)
  list_resp.append(tokenizer.decode(outputs[0], skip_special_tokens=False))

100%|██████████| 400/400 [03:26<00:00,  1.94it/s]


In [58]:
list_preds = []
for pred in list_resp:
  list_preds.append(extract_answer(pred, labels))

list_labels = []
for f in formatted_test:
  list_labels.append(f['output'].lower())

overall_accuracy = accuracy_score(list_labels, list_preds)
weighted_accuracy = balanced_accuracy_score(list_labels, list_preds)
print("Total Accuracy:", overall_accuracy)
print("Weighted Accuracy", weighted_accuracy)

Total Accuracy: 0.895
Weighted Accuracy 0.895


In [ ]:
# print(formatted_test[0])
# tokenized_test[0]

In [ ]:
# print(make_instruction_test(formatted_test[0]))
# print()
# print(f"Right answer is: {formatted_test[0]['output']}")

In [ ]:
# random.shuffle(formatted_train)

# # Печать первых 50 элементов
# for item in formatted_train[:50]:
#     print(make_instruction(item))
#     print('-------------------------------')